In [14]:
import pandas as pd
import re
from typing import Union
from rich.progress import track
import requests
from geopy.geocoders import ArcGIS

Change the datafile then everything should run fine! 😃

In [15]:
df = pd.read_csv('../data/CookCounty_Pharmacies.csv')
print(df.shape)
df.head()

(1725, 6)


,PharmacyName,PharmacyAddress,City,State,Zip,County
0,200 PHARMACY,9004 S Stony Island,CHICAGO,IL,60617,Cook County
1,666 DRUG CO,680 N LAKE SHORE DR,CHICAGO,IL,60611,Cook County
2,IVAN T MATTHEI PHARMACY,111 N WABASH ROOM 712,CHICAGO,IL,60602,Cook County
3,BRADYS PHARMACY,3235 W 111TH ST,CHICAGO,IL,60655,Cook County
4,WALGREENS,6798 N LINCOLN,LINCOLNWOOD,IL,60646,Cook County


Unless you have to change the source address fields 😅

In [16]:
def make_address(row) -> str:
    street = row['PharmacyAddress']
    city = row['City']
    state = row['State']
    zip_code = row['Zip']
    address = f"{street} {city} {state} {zip_code}".strip()
    return address

In [17]:
df['full_address'] = df.apply(lambda row: make_address(row), axis=1)
df['full_address'].sample(2)

896       5930 N. Broadway Chicago IL 60660
352    7523 W LAKE DR RIVER FOREST IL 60305
Name: full_address, dtype: object

In [22]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

geolocator = ArcGIS()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)
df['geo_location'] = df['full_address'].progress_apply(geocode)

100%|██████████| 1725/1725 [19:29<00:00,  1.47it/s]


In [23]:
df['coded_lat'] = df['geo_location'].apply(lambda x: x.latitude)
df['coded_long'] = df['geo_location'].apply(lambda x: x.longitude)
df['coded_score'] = df['geo_location'].apply(lambda x: x.raw.get('score'))

df.coded_score.describe()

count    1725.000000
mean       99.163078
std         2.345120
min        83.420000
25%        98.840000
50%       100.000000
75%       100.000000
max       100.000000
Name: coded_score, dtype: float64

In [24]:
interim = df.drop('geo_location', axis=1)
interim.to_csv('../data/geocoded_pharmacies.csv', index=False)